In [ ]:
from sbs_data_processing import data_processing as dp
from sbs_data_processing import data_conversion as dc
from sbs_data_processing import instrument_data as id
import pandas as pd
import numpy as np
from sbs_data_processing import util

pd.set_option('display.max_rows', None)

expected_data = id.cnv_to_instrument_data('./orca-test-data/SBE19/CAST0002/CAST0002_mod_filt_loop_percent.cnv')
data = id.cnv_to_instrument_data('./orca-test-data/SBE19/CAST0002/CAST0002_pd_mod.cnv')
window = 3
diff_length = int(window / data.interval_s)

# dp.loop_edit_depth(
#     depth=dc.depth_from_pressure(data.measurements['prSM'].values, data.latitude),
#     flag=data.measurements['flag'].values,
#     sample_interval=data.interval_s,
#     min_velocity_type=dp.MinVelocityType.PERCENT,
#     min_velocity=0.1,
#     window_size=window,
#     mean_speed_percent=20,
#     remove_surface_soak=False,
#     min_soak_depth=5,
#     max_soak_depth=20,
#     use_deck_pressure_offset=False,
#     exclude_flags=False
# )

pressure = data.measurements['prSM'].values
derived_depth = dc.depth_from_pressure(data.measurements['prSM'].values, 47.2778)
depth = data.measurements['depSM'].values
first_window = (np.diff(depth[0:diff_length+1]) / data.interval_s)
mean_speed = np.concatenate([np.zeros(diff_length), (depth[diff_length:] - depth[:-diff_length]) / diff_length / data.interval_s])
speed = np.concatenate([first_window, np.diff(depth[diff_length:], prepend=depth[diff_length-1]) / data.interval_s])

comp = {
    'pressure': pressure,
    'derived_depth': derived_depth,
    'depth': depth,
    'mean_speed': mean_speed,
    'speed': speed,
    'expected': expected_data.measurements['flag'].values,
    'result': data.measurements['flag'].values

}
util.plot(**comp)

print(sum(expected_data.measurements['flag'].values[0:-2] != data.measurements['flag'].values[0:-2]))

# error_rows = []
# for n, row in comp.iterrows():
#     if row['expected'] != row['result']:
#         error_rows.append(n)
# print(error_rows)

# comp[:]


In [ ]:
from sbs_data_processing import data_processing as dp
from sbs_data_processing import instrument_data as id
import pandas as pd
import numpy as np

a = np.array([1,4,1,1,3])
b = a > 1

max((v, n) for n, v in enumerate(a) if b[n])[1]



In [ ]:
from sbs_data_processing import data_processing as dp
from sbs_data_processing import data_conversion as dc
from sbs_data_processing import instrument_data as id
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', None)

a = id.cnv_to_instrument_data('./orca-test-data/SBE19/CAST0002/CAST0002_pd_mod.cnv').measurements['prSM'].values
b = id.cnv_to_instrument_data('./orca-test-data/SBE19/CAST0002/CAST0002_pd_mod_filt.cnv').measurements['prSM'].values
c = id.cnv_to_instrument_data('./orca-test-data/SBE19/CAST0002/CAST0002_pd_mod.cnv').measurements['depSM'].values
d = id.cnv_to_instrument_data('./orca-test-data/SBE19/CAST0002/CAST0002_pd_mod_filt.cnv').measurements['depSM'].values
x = range(len(a))

fig, ax = plt.subplots(figsize=(20,10))

ax.plot(x, a, x, b, x, c, x, d)
plt.show()

In [ ]:
from sbs_data_processing import instrument_data as id

raw = id.read_raw_tcp_from_hex('./orca-test-data/SBE19plusV2/E8001/E8001.hex')
print(raw)

In [ ]:
import numpy as np
import pandas as pd
from sbs_data_processing import instrument_data as id
from sbs_data_processing import data_processing as dp
from scipy import stats
import matplotlib.pyplot as plt

dataset = id.cnv_to_instrument_data('orca-test-data\SBE19plusV2\E8001\E8001.cnv')
pressure = dataset.measurements['prdM'].values
temperature = dataset.measurements['tv290C'].values
conductivity = dataset.measurements['c0S/m'].values
salinity = dataset.measurements['sal00'].values
flag = dataset.measurements['flag'].values

fig, ax = plt.subplots(figsize=(20,10))
x = range(len(pressure))
# ax.plot(x, pressure)

# SeaSoft bin averaging only considers data in up and/or down casts
# Using a modified loop_edit to get up/down cast info
cast = dp.loop_edit_pressure(
    pressure, dataset.latitude, flag, dataset.interval_s, dp.MinVelocityType.FIXED,
    0.25, 300, 20, True, 5, 20, False, False
)
ax.plot(x, cast)


# bins = np.histogram_bin_edges(pressure, 'auto')

# stat = stats.binned_statistic(pressure, [temperature, conductivity, salinity], 'mean')

# throwaway = 0 # for adding breakpoint

In [ ]:
import numpy as np
import pandas as pd
from sbs_data_processing import instrument_data as id
from sbs_data_processing import data_processing as dp
import matplotlib.pyplot as plt

expected_dataset = id.cnv_to_instrument_data('orca-test-data\SBE19plusV2\E8001\E8001_wild_edit.cnv')
expected_output = expected_dataset.measurements['prdM'].values

dataset = id.cnv_to_instrument_data('orca-test-data\SBE19plusV2\E8001\E8001.cnv')
pressure = dataset.measurements['prdM'].values
flags = dataset.measurements['flag'].values

wild_edit_output = dp.wild_edit(pressure, flags, 1, 3, 100, 0, False)

print((wild_edit_output != expected_output).sum())

# df = pd.DataFrame([wild_edit_output, expected_output]).transpose()
# print(df)

# for n in range(len(wild_edit_output)):
    # if wild_edit_output[n] != expected_output[n]:
    # print(f'{n}: {pressure[n]}, {wild_edit_output[n]}, {expected_output[n]}')

# fig, ax = plt.subplots(figsize=(20,10))
# x = range(len(pressure))
# ax.plot(x, pressure, x, wild_edit_output, x, expected_output)
# plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sbs_data_processing import instrument_data as id
from sbs_data_processing import data_processing as dp
import matplotlib.pyplot as plt

dataset = id.cnv_to_instrument_data('orca-test-data\SBE19plusV2\E8001\E8001.cnv')
pressure = dataset.measurements['prdM'].values
flags = dataset.measurements['flag'].values

expected_dataset = id.cnv_to_instrument_data(f'orca-test-data\SBE19plusV2\E8001\E8001_window_filter_boxcar_5.cnv')
pressure_expected = expected_dataset.measurements['prdM'].values

window_width = 5

pressure_filtered = dp.window_filter(pressure, flags, 'boxcar', window_width)

start = int(window_width/2)
end = int(0-window_width/2)

print((pressure_filtered[start:end] != pressure_expected[start:end]).sum())

for n in range(start,end):
    if pressure_filtered[n] != pressure_expected[n]:
        print(f'{n}: {pressure[n]}, {pressure_filtered[n]}, {pressure_expected[n]}')

# fig, ax = plt.subplots(figsize=(20,10))
# x = range(len(pressure))
# ax.plot(x, pressure, x, pressure_filtered, x, pressure_expected)
# plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sbs_data_processing import instrument_data as id
from sbs_data_processing import data_processing as dp
import matplotlib.pyplot as plt
from sbs_data_processing import util


filter_type = dp.WindowFilterType.COSINE
filter_settings = '5_excluded'

cnvdata = id.cnv_to_instrument_data('.\orca-test-data\SBE19plusV2\E8001\E8001_window_filter.cnv')
pressure = cnvdata.measurements['prdM'].values
flags = cnvdata.measurements['flag'].values

expected_dataset = id.cnv_to_instrument_data(
    f'.\orca-test-data\SBE19plusV2\E8001\E8001_window_filter_{filter_type.value}_{filter_settings}.cnv'
)
pressure_expected = expected_dataset.measurements['prdM'].values

window_width = 5

pressure_filtered = dp.window_filter(
    pressure, flags, filter_type, window_width, cnvdata.interval_s, 1, 0.25, True
)

pressure_rounded = np.round(pressure_filtered, 3)

# if not util.close_enough(pressure_filtered, pressure_expected, 3, 1e-12):
#     print('fail')
    
# for n in range(len(pressure)):
#     if not util.close_enough(pressure_filtered, pressure_expected, 3, 1e-12):
#         print(f'{n}: {pressure[n]}, {pressure_expected[n]}, {pressure_rounded[n]}, {pressure_filtered[n]}')

print(util.percent_match(pressure_rounded, pressure_expected))

util.plot(
    pressure = pressure,
    filtered = pressure_filtered,
    rounded = pressure_rounded,
    expected = pressure_expected,
)


In [ ]:
import numpy as np
from scipy import signal
from sbs_data_processing import instrument_data as id
from sbs_data_processing import data_processing as dp
from sbs_data_processing import util

# SETUP
test_data = id.cnv_to_instrument_data('.\orca-test-data\SBE19plusV2\E8001\E8001_window_filter.cnv')
data_in = test_data.measurements['tv290C'].values
window_type = dp.WindowFilterType.BOXCAR
window_width = 5

# WINDOW FILTER
if window_type == dp.WindowFilterType.BOXCAR:
    window = signal.boxcar(window_width)

elif window_type == dp.WindowFilterType.COSINE:
    window = signal.cosine(window_width)

elif window_type == dp.WindowFilterType.TRIANGLE:
    window = signal.triang(window_width)

elif window_type == dp.WindowFilterType.GAUSSIAN:
    window = signal.gaussian(window_width, 1/np.sqrt(2*np.log(2)))

else:
    window = np.zeros(window_width)

window /= window.sum()

data_out = np.convolve(data_in, window, 'same')

# VALIDATE
expected_data = id.cnv_to_instrument_data(
    f'.\orca-test-data\SBE19plusV2\E8001\E8001_window_filter_{window_type.value}_5.cnv'
)
expected_temperature = expected_data.measurements['tv290C'].values

print(util.percent_match(np.round(data_out, 4), expected_temperature))

util.plot(
    data_in = data_in,
    data_out = data_out,
    expected_temperature = expected_temperature,
)